
# **Gravitational Wave Analysis** Part 1
### SHUBHAM 2K19/EP/093
---
In this project, I'll use some python prebuilt module to analyse the gravitational wave data which are in the form of datasets and event files. These datasets and event files are much bigger that if one downloads and imports into their HDD/SSD stroge type then it can take some time to be imported in the notebook. So, I'm using the Google Colaboratory where the data is already stored at the servers and can be easily imported. This notebook uses Google's servers so, all the modules of python and commands will run on the server.



---

## Installing the required modules in the notebook

*   gwosc
*   gwpy


***

In [ ]:
! pip install -q 'gwosc==0.5.3'
! pip install -q gwpy

     |████████████████████████████████| 1.4 MB 5.5 MB/s 
     |████████████████████████████████| 51 kB 5.8 MB/s 
     |████████████████████████████████| 11.2 MB 35.4 MB/s 
     |████████████████████████████████| 890 kB 51.5 MB/s 
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 3.6 MB 33.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import gwosc
print("The version of GWOSC module we installed is", gwosc.__version__)

The version of GWOSC module we installed is 0.5.3


---
## Importing the required modules
***
Two things from the gwosc will help us query the datasets and events
* Module `gwosc.catalog` provides methods to search for some event in catalog
* Module `gwosc.datasets` provides methods to search datasets including full run datasets.

In [ ]:
from gwosc import datasets as gd
from gwosc import catalog as ct

Now, let's print al the events and the large strain datasets

In [ ]:
gwtc1 = ct.events('GWTC-1-confident')
print("The GWTC-1-confident events are:")
print(*gwtc1)

The GWTC-1-confident events are:
GW150914 GW151012 GW151226 GW170104 GW170608 GW170729 GW170809 GW170814 GW170817 GW170818 GW170823


In [ ]:
runs = gd.find_datasets(type = 'run')
print("The large strain datasets are:")
print(*runs)

The large strain datasets are:
BKGW170608_16KHZ_R1 O1 O1_16KHZ O2_16KHZ_R1 O2_4KHZ_R1 O3a_16KHZ_R1 O3a_4KHZ_R1 O3b_16KHZ_R1 O3b_4KHZ_R1 S5 S6


*** 
### Filtering the results by providing more arguments
***
The function `events()`, which we can call by the object `gd` we created while importing the module, takes more optional arguments to narrow down the results as per requirement. These arguments are `detector` and `segment`.  
* Keyword `detector` takes the value of the detector name by which we want to search the field.
* Keyword `segment` takes the value of the runtime interval as a tuple: the start time and stop time in GPS time format. \
ex. `segment=(tstart, tstop)`

In [ ]:
print(ct.events('GWTC-1-confident', detector='L1', segment=(1164556817, 1187733618))) 
# we can also use * operator in print statement to reference only the elements of the array shown below.

['GW170104', 'GW170608', 'GW170729', 'GW170809', 'GW170814', 'GW170817', 'GW170818', 'GW170823']


---
### Quering the GPS Time
---
Using the function `event_gps()`, we can query for the GPS Time for any specific event in the catalog. We can call this function by the object `gd` we created while importing the module.\
Input any name of the event that we got so far from the above queries, in the function.\
This GPS time is the time in seconds that have elapsed since the start of the GPS Epoch at midnight (00:00) on January 6th, 1980.

In [ ]:
GPS_time = gd.event_gps('GW150914')
print(GPS_time)

1126259462.4


One can convert the GPS time format into regular time format by using `gwpy.time.tconvert()`. \
Since, we haven't imported the module yet, so, let's import the module and convert the above GPS time format into regular format.

In [ ]:
import gwpy.time as gt
# Now, convert the GPS time interval for GWTC-1-confident as shown above
tStart, tStop = 1164556817, 1187733618
tStartConv = gt.tconvert(1164556817)
tStopConv = gt.tconvert(1187733618)

evt = ct.events('GWTC-1-confident', detector = 'L1', segment = (1164556817, 1187733618))
print(f"The events that ran from {tStartConv} to {tStopConv} are: ")
print(evt)

The events that ran from 2016-11-30 16:00:00 to 2017-08-25 22:00:00 are: 
['GW170104', 'GW170608', 'GW170729', 'GW170809', 'GW170814', 'GW170817', 'GW170818', 'GW170823']


We can query for the GPS Time interval for any observing run:\
For doing so, we can use the function `run_segment(run, host = api.DEFAULT_URL)`\
We can call this function by the object `gd`
> The argument `run` is the name of the large run dataset.\
> The argument `host` is optional.



In [ ]:
print(gd.run_segment('O1'))

(1126051217, 1137254417)


Now, print the run time interval for all large run observations.

In [ ]:
for data in runs:
    print("{:<20}: ({:<}, {:<})".format(data, *gd.run_segment(data)))

BKGW170608_16KHZ_R1 : (1180911618, 1180982427)
O1                  : (1126051217, 1137254417)
O1_16KHZ            : (1126051217, 1137254417)
O2_16KHZ_R1         : (1164556817, 1187733618)
O2_4KHZ_R1          : (1164556817, 1187733618)
O3a_16KHZ_R1        : (1238166018, 1253977218)
O3a_4KHZ_R1         : (1238166018, 1253977218)
O3b_16KHZ_R1        : (1256655618, 1269363618)
O3b_4KHZ_R1         : (1256655618, 1269363618)
S5                  : (815155213, 875232014)
S6                  : (931035615, 971622015)


Now, try to print the interval in regular time format.

In [ ]:
def gps2Regular(tStart, tStop) -> tuple:
    return (gt.tconvert(tStart), gt.tconvert(tStop))


In [ ]:
for data in runs:
    (ti, tf) = gps2Regular(*gd.run_segment(data))
    print(f"{data:<20} ran from: ({ti} to {tf})")
    # print("{:<20} ran from ({} to {})".format(data, *gps2Regular(*gd.run_segment(data))))

BKGW170608_16KHZ_R1  ran from: (2017-06-07 23:00:00 to 2017-06-08 18:40:09)
O1                   ran from: (2015-09-12 00:00:00 to 2016-01-19 16:00:00)
O1_16KHZ             ran from: (2015-09-12 00:00:00 to 2016-01-19 16:00:00)
O2_16KHZ_R1          ran from: (2016-11-30 16:00:00 to 2017-08-25 22:00:00)
O2_4KHZ_R1           ran from: (2016-11-30 16:00:00 to 2017-08-25 22:00:00)
O3a_16KHZ_R1         ran from: (2019-04-01 15:00:00 to 2019-10-01 15:00:00)
O3a_4KHZ_R1          ran from: (2019-04-01 15:00:00 to 2019-10-01 15:00:00)
O3b_16KHZ_R1         ran from: (2019-11-01 15:00:00 to 2020-03-27 17:00:00)
O3b_4KHZ_R1          ran from: (2019-11-01 15:00:00 to 2020-03-27 17:00:00)
S5                   ran from: (2005-11-04 16:00:00 to 2007-10-01 00:00:00)
S6                   ran from: (2009-07-07 21:00:00 to 2010-10-20 15:00:00)


To see only the confident events in O1:



In [ ]:
O1_confident = ct.events('GWTC-1-confident', segment = gd.run_segment('O1'))
print(f"The confident events in O1 are {O1_confident}")

The confident events in O1 are ['GW150914', 'GW151012', 'GW151226']


In [ ]:
O1_marginal = ct.events('GWTC-1-marginal', segment = gd.run_segment('O1'))
print(f"The marginal events in O1 are {O1_marginal}")

The marginal events in O1 are ['151008', '151012A', '151116']


---
### Quering for data files
---
The module `gwosc.locate` provides a function `get_event_urls()` to find the URLs of the data files associated with a given dataset.\
One can get the URLs by using the name of the event.\
First import the module. 

In [ ]:
from gwosc import locate as gl 
url = gl.get_event_urls('GW150914')
print(url)

['https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_4KHZ_R1-1126259447-32.hdf5', 'https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_4KHZ_R1-1126257415-4096.hdf5', 'https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW150914/v3/L-L1_GWOSC_4KHZ_R1-1126259447-32.hdf5', 'https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW150914/v3/L-L1_GWOSC_4KHZ_R1-1126257415-4096.hdf5']


Let's create a list or a dictionary that will store the URLs for the confident events in O1.

In [ ]:
urls_O1 = dict()
for name in O1_events:
    urls_O1[name] = gl.get_event_urls(name)
urls_O1

{'GW150914': ['https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_4KHZ_R1-1126259447-32.hdf5',
  'https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_4KHZ_R1-1126257415-4096.hdf5',
  'https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW150914/v3/L-L1_GWOSC_4KHZ_R1-1126259447-32.hdf5',
  'https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW150914/v3/L-L1_GWOSC_4KHZ_R1-1126257415-4096.hdf5'],
 'GW151012': ['https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW151012/v3/H-H1_GWOSC_4KHZ_R1-1128678885-32.hdf5',
  'https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW151012/v3/H-H1_GWOSC_4KHZ_R1-1128676853-4096.hdf5',
  'https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW151012/v3/L-L1_GWOSC_4KHZ_R1-1128678885-32.hdf5',
  'https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW151012/v3/L-L1_GWOSC_4KHZ_R1-1128676853-4096.hdf5'],
 'GW151226': ['https://www.gw-

As it can be seen above, the function `get_event_urls()` returns all the files in the form of `list()`, associated with a given event. However, we can filter on an of these by using keyword arguments. for ex. to get the url for the 32-second file for the LIGO Livingston detector: 

In [ ]:
Url = gl.get_event_urls(O1_events[1], duration = 32, detector = 'L1')
print(f"URL for {O1_events[1]} for 32s is:\n {Url}")

URL for GW151012 for 32s is:
 ['https://www.gw-openscience.org/eventapi/json/GWTC-1-confident/GW151012/v3/L-L1_GWOSC_4KHZ_R1-1128678885-32.hdf5']


In [ ]:
O1_evt = gd.find_datasets('O1')

O1Events = dict()
for names in O1_evt:
    Evt = ct.events(names, segment=gd.run_segment('O1'))
    Events = dict()
    for evtnames in Evt:
        try:
            link = get_event_urls(evtnames)
            Events[evtnames] = link
        except:
            continue
    O1Events[names] = Events